# setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import pickle
import random
from typing import Sequence
from tempfile import mkdtemp

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline


pd.set_option('display.max_rows', 100)

# load raw 

In [3]:

from src.io import csv_loader_factory, PickleLoader

In [4]:
loader = csv_loader_factory('poleval')
data = loader.load()
data

INFO:2022-01-07 04:23:17:io:loading from csv: ./data/poleval19.csv
INFO:2022-01-07 04:23:17:io:loaded 11039 rows


,dataset,text_raw,tag
0,train,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
1,train,@anonymized_account @anonymized_account Brawo ...,0
2,train,"@anonymized_account @anonymized_account Super,...",0
3,train,@anonymized_account @anonymized_account Musi. ...,0
4,train,"Odrzut natychmiastowy, kwaśna mina, mam problem",0
...,...,...,...
11034,test,@anonymized_account @anonymized_account No to ...,0
11035,test,"@anonymized_account Olej jak kto sie ubiera, p...",0
11036,test,@anonymized_account to oczywiste byłyście dziś...,0
11037,test,@anonymized_account Duda może się przyjąć w bi...,1


In [67]:
data

,dataset,text_raw,tag
0,dataset,text_raw,tag
1,train,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
2,train,@anonymized_account @anonymized_account Brawo ...,0
3,train,"@anonymized_account @anonymized_account Super,...",0
4,train,@anonymized_account @anonymized_account Musi. ...,0
...,...,...,...
11035,test,@anonymized_account @anonymized_account No to ...,0
11036,test,"@anonymized_account Olej jak kto sie ubiera, p...",0
11037,test,@anonymized_account to oczywiste byłyście dziś...,0
11038,test,@anonymized_account Duda może się przyjąć w bi...,1


In [8]:
path = os.path.join('./data/', 'training_set_clean_only_tags.txt')
aa = pd.read_csv(path)
aa.head()

,0
0,0
1,0
2,0
3,0
4,0


In [46]:
folder = './data/'

path = os.path.join(folder, 'training_set_clean_only_text.txt')
with open(path, 'r', encoding="utf8") as f:
    train_text = f.readlines()
    train_text = pd.Series([t[:-1] for t in train_text])

path = os.path.join(folder, 'training_set_clean_only_tags.txt')
train_tags = pd.read_csv(path, encoding='UTF-8')
train_tags = train_tags['0']

train_data = pd.DataFrame(zip(train_text, train_tags), columns=['text_raw', 'tag'])
train_data['dataset'] = 'train'

path = os.path.join(folder, 'test_set_clean_only_text.txt')
with open(path, 'r', encoding="utf8") as f:
    test_text = f.readlines()
    test_text = pd.Series([t[:-1] for t in test_text])

path = os.path.join(folder, 'test_set_clean_only_tags.txt')
test_tags = pd.read_csv(path, encoding='UTF-8')
test_tags = test_tags['0']

test_data = pd.DataFrame(zip(test_text, test_tags), columns=['text_raw', 'tag'])
test_data['dataset'] = 'test'

pol_eval = pd.concat([train_data, test_data], axis=0, ignore_index=True)
pol_eval = pol_eval[['dataset', 'text_raw', 'tag']]

pol_eval.to_csv('./data/poleval19.csv', encoding='UTF-8', index=False)

In [57]:
list(pol_eval['text_raw'])

['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
 '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
 '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
 '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
 'Odrzut natychmiastowy, kwaśna mina, mam problem',
 'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
 '@anonymized_account No nie ma u nas szczęścia 😉',
 '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
 '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
 '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account',
 '@anonymized_account @anonymi

In [60]:
list(pd.read_csv('./data/poleval19.csv', encoding='UTF-8')['text_raw'])

['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
 '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
 '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
 '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
 'Odrzut natychmiastowy, kwaśna mina, mam problem',
 'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
 '@anonymized_account No nie ma u nas szczęścia 😉',
 '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
 '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
 '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account',
 '@anonymized_account @anonymi

In [63]:
pd.read_csv('./data/poleval19.csv', encoding='UTF-8')

,dataset,text_raw,tag
0,train,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
1,train,@anonymized_account @anonymized_account Brawo ...,0
2,train,"@anonymized_account @anonymized_account Super,...",0
3,train,@anonymized_account @anonymized_account Musi. ...,0
4,train,"Odrzut natychmiastowy, kwaśna mina, mam problem",0
...,...,...,...
11034,test,@anonymized_account @anonymized_account No to ...,0
11035,test,"@anonymized_account Olej jak kto sie ubiera, p...",0
11036,test,@anonymized_account to oczywiste byłyście dziś...,0
11037,test,@anonymized_account Duda może się przyjąć w bi...,1


In [12]:
path = os.path.join('./data/', 'training_set_clean_only_text.txt')
aa = pd.read_csv(path, sep=None)
aa.head()

c:\users\micha\desktop\proj\gensim\venv\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


ParserError: Expected 2 fields in line 5, saw 3

In [7]:
aa.sum()/len(aa)

0    0.084761
dtype: float64

In [ ]:

from src.io import csv_loader_factory, PickleLoader
from src.preproc import RawDataCleaner, TextCleaner, SimpleTokenizer